# Solving the SCF problem

For the work in this notebook we will stick with the aluminum setup introduced before with one additional hinge: We will allow to make the problem harder or easier, by forming a supercell.

- Since we are dealing with periodic problems there is no unique definition of the lattice. Clearly any duplication of the lattice along an axis is also a valid lattice.
- This is exactly what a **supercell** is: An $n$-fold repetition in one of the lattice axes.

The following code achieves this for aluminium:

In [ ]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=13.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = diagm(fill(a, 3))
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms = [Al => [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]]

    # Make supercell in pymatgen:
    # We convert our lattice to the conventions used in pymatgen
    # and then back ...
    mg_struct = pymatgen_structure(lattice, atoms)
    mg_struct.make_supercell([1, 1, repeat])
    lattice = load_lattice(mg_struct)
    atoms = [Al => [s.frac_coords for s in mg_struct.sites]];

    # Construct the LDA model and discretise
    model = model_LDA(lattice, atoms, temperature=1e-3)
    PlaneWaveBasis(model; Ecut, kgrid)
end

As we will see in this notebook the modelling of a system generally becomes harder if the system becomes larger. 

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
  
  
- The aim of many tricks I will show in this workbook is to ensure the **number of SCF iterations remains constant** when the system size increases.

## Gaining insight

In DFTK one can easily patch or extend the SCF procedure
by replacing parts of the code with custom callback functions.

We will use this in this notebook to construct our own SCF solver,
but without needing to worry about all the nasty bits
(proper normalisation, numerically stable formation of the density etc.).

Before we do that, let's demonstrate first how to use callbacks to extract information from a running SCF:

In [ ]:
using DFTK
using Plots

p = plot(yaxis=:log)  # Setup an empty plot canvas
density_differences = Float64[]
function plot_callback(info)    
    if info.stage == :finalize
        # When done with the SCF: Plot it!
        plot!(p, density_differences, label="|ρout - ρin|", markershape=:x)
    else
        # Just add the density difference of this step
        push!(density_differences, norm(info.ρout - info.ρin))
    end
        
    info  # Pass info through to allow callback chaining
end

# Chain the custom callback with the default one
# (printing the convergence table)
callback = DFTK.ScfDefaultCallback() ∘ plot_callback
    
# Run the SCF and show the plot
scfres = self_consistent_field(aluminium_setup(); tol=1e-12, callback=callback);
p

**Exercise:** Try making this problem harder by running on `aluminium_setup(2)` and `aluminium_setup(5)` (or higher if you can efford it). What do you observe in the plot?

**Hint:** This callback allows to read or modify the full state of the SCF iteration,
which is a valuable tool when debugging an SCF algorithm.
When working in the REPL or with scripts one of my favourite callbacks is
```julia
using Infiltrator
function infiltrate_callback(info)
    @infiltrate info.n_iter == 5
    info    
end
scfres = self_consistent_field(aluminium_setup(); tol=1e-6,
                               callback=infiltrate_callback);
```
to take you right to the SCF state at a surprising iteration.

Now we have all the tools in place ... let's do some numerics:

## Building our own SCF 1: Fixed-point iterations

As we saw before the self-consistent field procedure required to solve the DFT problem can be written as a fixed-point problem
$$ F(\rho) = \rho $$
where $F(\rho) = D(V(\rho))$ is the basic SCF step. That is the construction of the Kohn-Sham Hamiltonian $H(\rho)$ given the density $\rho$, followed its diagonalisation to obtain its eigenpairs $(\varepsilon_{k i}, \psi_{ki})$
and from these a new density
$$ \rho(r) = \sum_{k\in\text{BZ}} \sum_i \psi_{ki}(r) \, \psi_{ki}^\ast(r).$$

We will not be concerned with $F$ itself, which we will take for "granted" (i.e. delivered by DFTK).
What we will consider, however, is multiple ways to solve the DFT fixed-point problem.

The easiest are plain **fixed-point iterations**, i.e.

$$ \rho_{n+1} = F(\rho_n), $$
starting from a hopefully good initial guess $\rho_0$. DFTK automatically provides a reasonable
guess density, such that we only need to take care of the iterations themselves.
In the language of DFTK this algorithm is written as:

In [ ]:
function fixed_point_iteration(F, ρ₀, maxiter; tol)
    # F:        The SCF step function
    # ρ₀:       The initial guess density
    # maxiter:  The maximal number of iterations to be performed
    # tol:      The selected convergence tolerance
    
    ρ  = ρ₀
    Fρ = F(ρ)
    for n = 1:maxiter 
        # If change less than tolerance, break iterations:
        if norm(Fρ - ρ) < tol
            break
        end
        ρ  = Fρ
        Fρ = F(ρ)
    end
    
    # Return some stuff DFTK needs ...
    (fixpoint=ρ, converged=norm(Fρ - ρ) < tol)
end;

# use this algorithm inside DFTK's SCF for solving the silicon problem
# (the other parameters are needed to overwrite some DFTK defaults
#  we don't want to use just yet).
self_consistent_field(aluminium_setup(); solver=fixed_point_iteration, damping=1.0, maxiter=40);

As can be observed this algorithm is not very good and completely fails to converge. This is a known limitation of this algorithm, which is why it is not used in practice.

## Step 2: Damped iterations

The next step is to introduce a so-called damping parameter $\alpha$, which is given a value between $0$ and $1$. One now iterates as follows:
$$ \rho_{n+1} = \rho_{n} + \alpha (F(\rho_n) - \rho_n) $$
In other words the update $F(\rho_n) - \rho_n$ proposed in the $n$-th SCF step is not fully taken, but scaled-down by the damping $\alpha$.

**Exercise:** Modify `fixed_point_iteration` such that it supports this *damped* fixed-point iteration. Try different values for $\alpha$ between $0$ and $1$ and estimate roughly the $\alpha$ which gives fastest convergence. For which $\alpha$ do you observe no convergence at all?

**Note:** The observations you make here are general. We will argue in the next notebook why every SCF converges (locally) if a small enough $\alpha > 0$ is chosen.

## Step 3: Anderson acceleration

The `fixed_point_iteration` function above (with the damping extension) already covers the main gist of standard DFT algorithms. To make things converge faster the next step to follow is Anderson acceleration, where not only $\rho_n$ and $F(\rho_n)$, but also older iterates are used to propose the next density.

For Anderson one exploits that the update $R(\rho) = F(\rho) - \rho$ is also the residual of the fixed-point problem $F(\rho) = \rho$, i.e. how far away we are from the fixed-point density. A good next density $\rho_{n+1}$ therefore should be found by minimising an approximation for $R(\rho_{n+1})$. Assuming the SCF was linear in the density (which it is not), a good idea is to find a linear combination of residuals
$$\sum_i \beta_i R(\rho_i)$$
which has the smallest possible norm and to use these coefficients $\beta_i$ to extrapolate the next
density
$$ \rho_{n+1} =  \sum_i \beta_i (\rho_i + \alpha R(\rho_i)) $$
where you notice the "standard" damped fixed-point iteration in the summed terms.

In terms of an algorithm this is 

In [ ]:
function anderson_iteration(F, ρ₀, maxiter; tol)
    # F:        The SCF step function
    # ρ₀:       The initial guess density
    # maxiter:  The maximal number of iterations to be performed
    # tol:      The selected convergence tolerance
    
    converged = false
    ρ  = ρ₀
    ρs = []
    Rs = []
    for n = 1:maxiter
        Fρ = F(ρ)
        Rρ = Fρ - ρ
        converged = norm(Rρ) < tol
        converged && break
        
        ρnext = vec(ρ) .+ vec(Rρ)
        if !isempty(Rs)           
            # will be developed in the workshop ...
        end
                    
        push!(ρs, vec(ρ))
        push!(Rs, vec(Rρ))
        ρ = reshape(ρnext, size(ρ₀)...)
    end
    
    # Return some stuff DFTK needs ...
    (fixpoint=ρ, converged=converged)
end

To work with this algorithms we will use DFTK's intrinsic mechanism to choose a damping. The syntax for this is

```julia
repeat = 1
self_consistent_field(aluminium_setup(repeat);
                      solver=anderson_iteration,
                      damping=0.8, maxiter=40);
```
to choose a damping of $\alpha = 0.8$ and run for at most `maxiter` iterations.

**Exercise:** Pick $\alpha = 0.8$ and make the problem harder by increasing `repeat` (e.g. `2`, `4`, `6`, `8`). Can you make Anderson fail to converge? What do you notice in terms of the number of iterations and runtimes?

In [ ]:
# Your solution here

## Step 4: Using NLsolve

Of course it is never a good idea to recode standard algorithms such as Anderson acceleration when they are already implemented in other Julia packages.
In DFTK we actually rely on the Anderson solver from [NLsolve.jl](https://github.com/JuliaNLSolvers/NLsolve.jl), which is even a bit smarter than the simple version we coded up above.

[Our code](https://github.com/JuliaMolSim/DFTK.jl/blob/master/src/scf/scf_solvers.jl#L16--L23) to invoke NLsolve is pretty much just:

In [ ]:
using NLsolve
function nlsolve_solver(F, ρ₀, maxiter; tol)
    res = nlsolve(ρ -> F(ρ) - ρ, ρ₀; method=:anderson, m=10, xtol=tol,
                  ftol=0.0, show_trace=false, iterations=maxiter)
    (fixpoint=res.zero, converged=converged(res))
end

self_consistent_field(basis; solver=nlsolve_solver, damping=0.8, maxiter=40, tol=1e-6);

## Step 5: Using preconditioned iterations

Anderson allows us to push the boundary for SCF methods, but for larger or more challenging systems it is not fully sufficient. The next ingredient for a stable SCF procedure is based on the insight we already had previously: The convergence properties of an SCF depends on the material which is modelled.
As a result the ideal SCF procedure should be slightly different for each material.
In our discussion so far we did not yet take that into account.

The standard approach to include material specificity into the SCF is to employ *preconditioned* damped fixed-point iterations.
To explain the idea, let us consider again a framework without Anderson acceleration.
Preconditioned iterations are then
$$ \rho_{n+1} = \rho_{n} + \alpha P^{-1} (F(\rho_n) - \rho_n), $$
where $P^{-1}$ is a preconditioner that hopefully improve convergence.
To re-introduce Anderson around this iteration
just replace the previous definition of $R$ by
$R(\rho) = P^{-1} (F(\rho_n) - \rho_n)$, that's it.

As we will discuss in the next notebook the ideal preconditioning $P$
depends on the dielectric properties of the material (e.g. whether it is a
metal, insulator or semiconductor as well as other details). 
Finding a good $P$ is not always easy and for some cases satisfactory options are not yet known. For our aluminium case, however, we are lucky. The `KerkerMixing` implemented in DFTK provides a good $P$ for aluminium.

You might wonder about the term *mixing*. In an interdisciplinary community like DFT, different scientists use different vocabulary and "mixing" is the "physicists' term" used for preconditioning.

To use `KerkerMixing` with DFTK run the SCF as follows
```julia
self_consistent_field(basis; α=0.8, mixing=KerkerMixing());
```


**Exercise:** Try this setup for different values of `repeat` and check the number of iterations needed. Other mixings DFTK has to offer are `DielectricMixing` (best for semiconductors) or `LdosMixing` (best for metal-insulator-mixtures). Try them as well and summarise your findings.


You should notice that choosing a preconditioner matching the material under study aids a fast SCF convergence, but that sometimes being off does not seem to do much harm for our case. In the next notebook we will introduce some tools to analyse this more systematically.

#### Takeaways:
- Large systems require a matching preconditioner to converge in few SCF iterations
- Anderson acceleration and/or small damping aids convergence
- Provided Anderson is used one often gets away using a non-matching preconditioner